In [2]:
from snowflake.snowpark import Session
import configparser
def get_session():
    parser = configparser.ConfigParser()
    # Add the credential file name here
    parser.read('config.ini')

    connection_params = dict(user=parser['Credentials']['user'], 
                         password=parser['Credentials']['password'], 
                         account=parser['Credentials']['account'], 
                         warehouse=parser['Credentials']['warehouse'], 
                         database=parser['Credentials']['database'],
                         schema=parser['Credentials']['schema'], 
                         role=parser['Credentials']['role'])

    session = Session.builder.configs(connection_params).create()
    return session

In [3]:
session = get_session()

### Deploy the model from stage location as a UDF

In [4]:
session.clear_imports()
session.clear_packages()
from snowflake.snowpark.functions import udf,call_udf

# Add trained model as dependency
session.add_import("@models/Predictive_Maintenance_model_20230919_194031.joblib.gz")
import pandas
@udf(name='predict', session=session,is_permanent=True,replace=True,stage_location="@SCORE",packages=["snowflake-snowpark-python","pandas", "joblib","scikit-learn"])
def score(payload: list) -> str:
    import sys
    import pandas as pd
    from joblib import load

    IMPORT_DIRECTORY_NAME = "snowflake_import_directory"
    import_dir = sys._xoptions[IMPORT_DIRECTORY_NAME]

    pipeline_file = import_dir + 'Predictive_Maintenance_model_20230919_194031.joblib.gz'
    pipeline = load(pipeline_file)

    prediction = pipeline.predict([payload])[0]
    
    return prediction

The version of package joblib in the local environment is 1.3.2, which does not fit the criteria for the requirement joblib. Your UDF might not work when the package version is different between the server and your local environment


#### Sample Input 

In [5]:
inp =[6,407438,1,3,'S1F0',1,0,26953834,0,0,2.079441541679836,0.0,3.9702919135521215,0.0,2.079441541679836]

Consuming the model one request at a time. For batch inferenece, deploy the model as vectorized UDF

In [6]:
output = session.sql("select PREDICT("+str(inp)+")").collect()

SnowparkSQLException: (1304): 01af1d16-0503-cace-0072-f3030bf67b5a: 100357 (P0000): Python Interpreter Error:
Traceback (most recent call last):
  File "_udf_code.py", line 49, in compute
  File "_udf_code.py", line 38, in wrapper
  File "<ipython-input-4-03234398b113>", line 20, in score
NameError: name 'df' is not defined
 in function PREDICT with handler compute

In [ ]:
output

In [ ]:
    #add all imports
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.metrics import recall_score, f1_score, roc_auc_score, confusion_matrix,classification_report
    from sklearn.linear_model import SGDClassifier
    from sklearn.ensemble import RandomForestClassifier
    
    from sklearn.pipeline import Pipeline
    
    df_final = session.sql("SELECT * from {table}".format(table="predictive_maintenance_final")).to_pandas()
    
    # prepare data for training
    X = df_final.drop(['FAILURE'], axis = 1)
    y = df_final['FAILURE']
    X_train, X_test, Y_train, Y_test = train_test_split(X,y,random_state = 0, test_size=0.25, shuffle = True)
    X_train.reset_index(inplace = True, drop = True)
    Y_train.reset_index(inplace = True, drop = True)

    X_test.reset_index(inplace = True, drop = True)
    Y_test.reset_index(inplace = True, drop = True)
    
    x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, random_state = 0, test_size=0.25)
    
    pipeline = Pipeline([
        ('OHE', OneHotEncoder(handle_unknown = 'ignore',sparse=False)),
#         ('SCALER', StandardScaler()),
        ('SGD', RandomForestClassifier(random_state=0))
     ])
    
    pipeline.fit(x_train,y_train)
    y_pred = pipeline.predict(x_val) 
    cf_matrix = confusion_matrix(y_val, y_pred)
    cls_report = classification_report(y_val, y_pred, output_dict=True)
    print(cls_report)

In [103]:
pred = pipeline.predict(X_test)
pred["probabilities"] = pipeline.predict_proba(X_test)[["predict_proba_True"]]
X_test["PREDICTION"] = pred["PREDICTION"]
X_test["PROBABILITIES"] = pred["probabilities"]
X_test["LABEL"] = Y_test

y_prob_r = np.array(X_test["PROBABILITIES"])
y_test_r = pred["CHURN"]
y_pred_r = pred["PREDICTION"]
X_test = X_test.drop(["CHURN", "PREDICTION", "probabilities"], axis=1)
# y_train = train["CHURN"]

AttributeError: probability estimates are not available for loss='hinge'

In [67]:
import pickle
pickle.dump(pipeline, open("/data/pipeline.pkl", 'wb'))

In [68]:
loaded_pipeline = pickle.load(open("/data/pipeline.pkl", 'rb'))

Write the score function

In [71]:
from refractml import *
from refractml.constants import MLModelFlavours
import requests

/tmp/pip_packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [72]:
@scoring_func
def score(model, request):
    payload = request.json["payload"]
    prediction = pipeline.predict([payload])[0]
    return prediction

In [74]:
req = requests.Request()
req.json = {"payload":[6,407438,1,3,'S1F0',1,0,26953834,0,0,2.079441541679836,0.0,3.9702919135521215,0.0,2.079441541679836]}
print(score(loaded_pipeline,req))

0


/opt/conda/lib/python3.8/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


### Register on refract

In [101]:
## registering the model in refract.
register_model(loaded_pipeline, 
               score, 
               name="Predictive_Maintenance", 
               description="By analyzing data patterns and using machine learning algorithms, it predicts when maintenance is needed, allowing organizations to perform repairs or servicing precisely when required, minimizing downtime, reducing costs, and preventing unexpected equipment failures. This approach enhances operational efficiency and extends the lifespan of assets by addressing maintenance needs based on data-driven insights rather than fixed schedules.",
               flavour=MLModelFlavours.sklearn,
               model_type="classification",
               y_true=y_val,
               y_pred=y_pred, 
               features=x_train.columns,
               labels=[0,1],
#                input_type="json", 
               explain_ai=True,
               x_train=x_train, 
               x_test=x_val, 
               y_train=y_train.tolist(),
               y_test=y_val.tolist(),
               feature_names=X_train.columns.tolist(),
               original_features=X_train.columns.tolist(),
               feature_ids=X_train.columns,)

ValueError: prob is/are missing